In [3]:
import pandas as pd
#import tensorflow as tf
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report
import numpy as np
from glob import glob
import joblib


In [4]:
df_nonflag = []
df_flag =[]
df_nonflag = pd.concat(
    [pd.read_parquet(x) for x in glob('/mnt/22A63810A637E2C9/Code/Twitch-Spam-detection/Data/chats/chats_nonflag_*.parquet')],
    ignore_index=True)
df_flag = pd.concat(
    [pd.read_parquet(x) for x in glob('/mnt/22A63810A637E2C9/Code/Twitch-Spam-detection/Data/chats/chats_flagged_*.parquet')],
    ignore_index=True)

ytdata = pd.read_csv("/mnt/22A63810A637E2C9/Code/Twitch-Spam-detection/Data/DirtyWords.csv")
ytdata = ytdata.drop(columns=["language",'id'])
ytdata['spam'] = 1
ytdata = ytdata.rename(columns={"word": "body"})
yt1 = pd.read_csv("/mnt/22A63810A637E2C9/Code/Twitch-Spam-detection/Data/Youtube01-Psy.csv")
yt1 = yt1.drop(columns=['COMMENT_ID', 'AUTHOR', 'DATE',])
yt1 = yt1.rename(columns={"CLASS": "spam", "CONTENT": "body"})
yt2 = pd.read_csv("/mnt/22A63810A637E2C9/Code/Twitch-Spam-detection/Data/Youtube02-KatyPerry.csv")
yt2 = yt2.drop(columns=['COMMENT_ID', 'AUTHOR', 'DATE',])
yt2 = yt2.rename(columns={"CLASS": "spam", "CONTENT": "body"})
yt3 = pd.read_csv("/mnt/22A63810A637E2C9/Code/Twitch-Spam-detection/Data/Youtube03-LMFAO.csv")
yt3 = yt3.drop(columns=['COMMENT_ID', 'AUTHOR', 'DATE',])
yt3 = yt3.rename(columns={"CLASS": "spam", "CONTENT": "body"})
yt4 = pd.read_csv("/mnt/22A63810A637E2C9/Code/Twitch-Spam-detection/Data/Youtube05-Shakira.csv")
yt4 = yt4.drop(columns=['COMMENT_ID', 'AUTHOR', 'DATE',])
yt4 = yt4.rename(columns={"CLASS": "spam", "CONTENT": "body"})
yt5 = pd.read_csv("/mnt/22A63810A637E2C9/Code/Twitch-Spam-detection/Data/Youtube04-Eminem.csv")
yt5 = yt5.drop(columns=['COMMENT_ID', 'AUTHOR', 'DATE',])
yt5 = yt5.rename(columns={"CLASS": "spam", "CONTENT": "body"})
ytprof = pd.read_csv("/mnt/22A63810A637E2C9/Code/Twitch-Spam-detection/Data/profanity_en.csv")


df = pd.concat([df_flag, df_nonflag], axis=0, ignore_index=True)
df = df.drop_duplicates()

In [5]:
spam_labels = ['hidden','deleted']
df['spam'] = df['label'].apply(lambda x: 1 if x in spam_labels else 0)
df=df.drop(columns=['label'])
dataframes = [ytdata, yt1, yt2, yt3, yt4, yt5]
c_df=pd.concat(dataframes,ignore_index=True)
X_train, X_tes, y_train, y_test = train_test_split(
    c_df['body'], c_df['spam'], test_size=0.2, random_state=69)
randomized_df = df.sample(frac=1.0, random_state=64)
randomized_df.head()

,body,spam
6263746,めんどくせぇなｗ,0
4988033,あまりに強すぎた,0
85982,@プランプ I agree with Coco that the best course o...,1
3978432,kusojapan timeeee,0
845092,聴診器ですかｗ,1


In [6]:
print("Total data : ", len(c_df))

Total data :  4155


In [7]:
print(ytdata.columns)
print(yt3.columns)
print(yt1.columns)
print(df.columns)
print(yt2.columns)
print(yt4.columns)
print(yt5.columns)

Index(['body', 'spam'], dtype='object')
Index(['body', 'spam'], dtype='object')
Index(['body', 'spam'], dtype='object')
Index(['body', 'spam'], dtype='object')
Index(['body', 'spam'], dtype='object')
Index(['body', 'spam'], dtype='object')
Index(['body', 'spam'], dtype='object')


In [8]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_tes)
SClassifier=MultinomialNB()
SClassifier.fit(X_train_tfidf,y_train)
pred =SClassifier.predict(X_test_tfidf)
report = classification_report(y_test,pred)
print("Spam Classification Report:")
print(report)
print("Naive Bayes model accuracy(in %):",accuracy_score(y_test, pred)*100)

Spam Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.63      0.76       199
           1       0.89      0.99      0.94       632

    accuracy                           0.91       831
   macro avg       0.93      0.81      0.85       831
weighted avg       0.91      0.91      0.90       831

Naive Bayes model accuracy(in %): 90.61371841155234


In [9]:
svm_classifier = SVC(kernel='linear') 
# poly was the lowest with 86% linear is highest then sigmoid and then rbf.
svm_classifier.fit(X_train_tfidf, y_train)
y_pred_svm = svm_classifier.predict(X_test_tfidf)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
report_svm = classification_report(y_test, y_pred_svm)
print(f"Accuracy: {accuracy_svm:.2f}")
print(report_svm)
print("SVM model accuracy(in %):",accuracy_score(y_test, y_pred_svm)*100)

Accuracy: 0.94
              precision    recall  f1-score   support

           0       0.92      0.80      0.86       199
           1       0.94      0.98      0.96       632

    accuracy                           0.94       831
   macro avg       0.93      0.89      0.91       831
weighted avg       0.94      0.94      0.93       831

SVM model accuracy(in %): 93.62214199759326


In [10]:
rf_classifier = RandomForestClassifier(n_estimators=100) 
rf_classifier.fit(X_train_tfidf, y_train)
y_pred_rf = rf_classifier.predict(X_test_tfidf)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
report_rf = classification_report(y_test, y_pred_rf)
print(f"Accuracy: {accuracy_rf:.2f}")
print(report_rf)
print("Random Forest model accuracy(in %):",accuracy_score(y_test, y_pred_rf)*100)

Accuracy: 0.94
              precision    recall  f1-score   support

           0       0.89      0.85      0.87       199
           1       0.95      0.97      0.96       632

    accuracy                           0.94       831
   macro avg       0.92      0.91      0.92       831
weighted avg       0.94      0.94      0.94       831

Random Forest model accuracy(in %): 94.1034897713598


In [11]:
logistic_classifier = LogisticRegression()
logistic_classifier.fit(X_train_tfidf, y_train)
y_pred_lr = logistic_classifier.predict(X_test_tfidf)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
report_lr = classification_report(y_test, y_pred_lr)
print(f"Accuracy: {accuracy_lr:.2f}")
print(report_lr)

Accuracy: 0.90
              precision    recall  f1-score   support

           0       0.95      0.59      0.73       199
           1       0.89      0.99      0.94       632

    accuracy                           0.90       831
   macro avg       0.92      0.79      0.83       831
weighted avg       0.90      0.90      0.89       831



In [17]:
#tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')
comment1 = ['go kill yourself']
comment1_vect = tfidf_vectorizer.transform(comment1)
rf_classifier.predict_proba(comment1_vect)[:,1]

array([0.76063355])

In [ ]:
import numpy as np
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *

# Constants
WINDOW_SIZE = 600
GRID_SIZE = 50
AMPLITUDE = 0.1
DAMPING = 0.99
UPDATE_INTERVAL = 10  # milliseconds

# Global variables
grid = np.zeros((GRID_SIZE, GRID_SIZE), dtype=float)
velocity = np.zeros((GRID_SIZE, GRID_SIZE), dtype=float)

def init():
    glClearColor(0.0, 0.0, 0.0, 1.0)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluOrtho2D(0, WINDOW_SIZE, 0, WINDOW_SIZE)

def update_wave(value):
    global grid, velocity
    new_grid = np.zeros((GRID_SIZE, GRID_SIZE), dtype=float)
    new_velocity = np.zeros((GRID_SIZE, GRID_SIZE), dtype=float)

    for i in range(1, GRID_SIZE - 1):
        for j in range(1, GRID_SIZE - 1):
            new_velocity[i][j] = (
                velocity[i][j] * DAMPING
                + (grid[i + 1][j] + grid[i - 1][j] + grid[i][j + 1] + grid[i][j - 1]) * 0.25
                - grid[i][j]
            )
            new_grid[i][j] = grid[i][j] + new_velocity[i][j]

    grid, velocity = new_grid, new_velocity
    glutPostRedisplay()
    glutTimerFunc(UPDATE_INTERVAL, update_wave, 0)

def display():
    glClear(GL_COLOR_BUFFER_BIT)
    glColor3f(1.0, 1.0, 1.0)
    glBegin(GL_POINTS)
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            glVertex2f(i * (WINDOW_SIZE / GRID_SIZE), j * (WINDOW_SIZE / GRID_SIZE) + grid[i][j])
    glEnd()
    glFlush()

def main():
    glutInit(sys.argv)
    glutInitDisplayMode(GLUT_SINGLE | GLUT_RGB)
    glutInitWindowSize(WINDOW_SIZE, WINDOW_SIZE)
    glutCreateWindow(b"Wave Simulator")
    glutDisplayFunc(display)
    glutTimerFunc(UPDATE_INTERVAL, update_wave, 0)
    init()
    glutMainLoop()

if __name__ == "__main__":
    main()
